In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# import json
# file = open('./../addresses.json')
# private_keys = json.load(file)
# print(private_keys[9]["private key"])
import json
file = open('./../r_addresses.json')
wallet = json.load(file)

In [4]:
SIGHASH_ALL = 1

In [5]:
# lock p2tr
from key import ECPubKey,ECKey
from messages import CTxIn,COutPoint,CTxOut,CTransaction,CTxWitness,CTxInWitness
from script import tagged_hash, sha256,taproot_construct,SegwitV0SignatureHash
import sys
sys.path.append("..")
from ecc import PrivateKey


prev_tx_hx = int("759fa5670209f5edb5d9e911ec02be35e099bd50f8f241696e404d77e3c36ca8", 16)
prv_amount=int(100*100000000)
# prv_script=bytes.fromhex("76a9148fe86e1e11732142fe7fc6671037f28406cc05fb88ac")
prv_pub_key_hash = "7d7143a28a3b836e3043f3ba5dbcdc4ee29b7457"
prv_script=bytes.fromhex(f"76a914{prv_pub_key_hash}88ac")
prev_index = 1

p=ECKey()
# p.set(0x012d606029786350ea1c8d1cedb44b7f82b0b095e86b291cb82ce845c0074889)
p.set(0x00db5cdc1663eea25406126e642f72ef7c78f68d249589d2873c613e61929eea)
# target_public_key_bytes=bytes.fromhex(private_keys[13]["public key"])
target_public_key_bytes=bytes.fromhex(wallet[12]["public key"])
target_amount = int(99.999*100000000)

tx_in =CTxIn(COutPoint(prev_tx_hx, prev_index))

target_public_key=ECPubKey()
target_public_key.set(target_public_key_bytes)

locking_script = taproot_construct(target_public_key)[0]

tx_outs = CTxOut(target_amount, locking_script)

tx = CTransaction()
tx.vin=[tx_in]
tx.vout=[tx_outs]
tx.calc_sha256()
tx_sig_hash = SegwitV0SignatureHash(prv_script,tx,0,0x01,prv_amount)
tx_input_0_wit = CTxInWitness([p.sign_ecdsa(tx_sig_hash) + (1).to_bytes(1, 'big'),p.get_pubkey().get_bytes(False)])
tx.wit.vtxinwit = [tx_input_0_wit]

print(tx.serialize_with_witness().hex())

# Use for Next Transaction
tx1_outs = tx.vout


01000000000101eb279edb342134ca732246f4a3e42df55eb225fbe57c4870f375850307174fe60100000000ffffffff01605d0a540200000022512068faa8de787e97e0b61386debad632bb0891b53cd58e7b897dbe5fe2d4e87c9102483045022100d5d0d84e83911260cb3e4bcaf909cdce57295fc11cb288d6b32f64468c999fe202201ecfc4f0a3a2fc09a305f852fc699db7f8b3bb750cb6812a46f8f63881440bb5012103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee00000000


In [26]:
# unlocking p2tr key path
from script import get_p2wpkh_script, TaprootSignatureHash
from segwit_addr import decode_segwit_address

taproot_locking_tx_id = int("6885cdf8c02e386a9bb1085e7826aa43dcfd6bddd6b6a9404b99ce6c4fdf2688", 16)
taproot_locking_tx_index = 0

# taproot_unlocking_sec_key = ECKey().set(bytes.fromhex("1f31014cced3d1a22262825455ce2b8d2bb988482184797c77fc4bcdbd64f271"))
taproot_unlocking_sec_key = ECKey().set(bytes.fromhex("13e4f9204068a232f23fe6305bb6c1fcb6d2c6e53347956a7fbaf2953737ab4c"))
taproot_unlocking_sec_key.negate()
target_amount = int(99.998*100000000)
target_b32_address = wallet[12]["address"]

tx_in_0 = CTxIn(COutPoint(taproot_locking_tx_id, taproot_locking_tx_index))
tx_ins = [tx_in_0]

locking_public_key_hash = bytes(decode_segwit_address("bcrt", target_b32_address)[1])
locking_script = get_p2wpkh_script(locking_public_key_hash)
tx_out_0 = CTxOut(target_amount, locking_script)
tx_outs = [tx_out_0]

tx = CTransaction()
tx.vin = tx_ins
tx.vout = tx_outs
tx.calc_sha256()

tweak = tagged_hash("TapTweak", taproot_unlocking_sec_key.get_pubkey().get_bytes())
tweaked_unlocking_secret_key = taproot_unlocking_sec_key.tweak_add(tweak)
prev_tx_outs = tx1_outs
tx_sig_hash_0 = TaprootSignatureHash(tx, prev_tx_outs, 0X00)
tweaked_unlocking_secret_key.negate()
tx_sign_0 = tweaked_unlocking_secret_key.sign_schnorr(tx_sig_hash_0)

tx_input_0_wit = CTxInWitness([tx_sign_0])
tx.wit.vtxinwit = [tx_input_0_wit]

print(tx.serialize_with_witness().hex())

010000000001018826df4f6cce994b40a9b6d6dd6bfddc43aa26785e08b19b6a382ec0f8cd85680000000000ffffffff01c0d6085402000000160014fce7aed87fbb8b90029dee7fe3b5ed28466347a50140e6129959c249592026bac52be9bc4f7e566927093725da96ef69ab32d9aaf2f80826428711e3a9177bde3273a7626902046184a195aa493f577610c3aef6559c00000000
